来源：https://tensorflow.google.cn/tutorials/keras/save_and_restore_models?hl=zh-cn

In [1]:
import os
import tensorflow as tf
from tensorflow import keras

获取示例数据集

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

# 定义模型

In [3]:
# Returns a short sequential model
def create_model():
# 配置序列模型
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.nn.softmax)
  ])
  
# 配置模型学习流程
  model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])

  return model

# Create a basic model instance
model = create_model()
# 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


# 训练期间保存检查点
__检查点__存在2大功能：
- 在训练期间，可以从上次暂停的地方继续训练，防止训练过程中段
- 训练结束时，可以使用__经过训练的模型__，而__无需重新训练该模型__

## 检查点回调用法
训练模型，并将`ModelCheckpoint`回调传递给该模型

___默认的 TensorFlow 格式仅保存最近的 5 个检查点___

In [4]:
# 创建检查点路径
cp_path='training_1/cp.ckpt'
cp_dir=os.path.dirname(cp_path)

# 创建检点回调,通过参数设置成只保存权重
cp_callback=tf.keras.callbacks.ModelCheckpoint(
                                               cp_path,
                                               save_weights_only=True,
                                               verbose=1
                                              )

# 训练模型，将回调传递给该模型
model.fit(
          train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback]
         )

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 6s 6ms/step - loss: 1.1815 - acc: 0.6570 - val_loss: 0.7268 - val_acc: 0.7860

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
1000/1000 [==============================] - 0s 176us/step - loss: 0.4273 - acc: 0.8850 - val_loss: 0.5840 - val_acc: 0.8160

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
1000/1000 [==============================] - 0s 244us/step - loss: 0.2863 - acc: 0.9310 - val_loss: 0.4638 - val_acc: 0.8580

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
1000/1000 [==============================] - 0s 182us/step - loss: 0.2027 - acc: 0.9540 - val_loss: 0.4643 - val_acc: 0.8490

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
1000/1000 [==============================] - 0s 199us/step - loss: 0.1682 - acc: 0.9620 - val_loss: 0.4218 - val_acc: 0.8640

Epoch 00005: saving model to training_1/cp.ckpt
Epoch 6/10
1000/1000 [=======

### 从权重恢复模型
恢复模型需知：
- 仅通过权重恢复模型时，模型必须与原始模型架构相同

#### 创建新模型，并进行评估

In [5]:
# 创建一个新模型
model_new=create_model()

# 评估该新模型
loss,acc=model_new.evaluate(test_images,test_labels)
print('untrained model acc{:.2%}'.format(acc))

1000/1000 [==============================] - 0s 150us/step
untrained model acc10.40%


#### 加载权重，并重新评估

In [6]:
# 加载权重
model.load_weights(cp_path)

#评估模型
loss,acc=model.evaluate(test_images,test_labels)
print('Restored model acc{:.2%}'.format(acc))

1000/1000 [==============================] - 0s 46us/step
Restored model acc10.40%


#### 新模型-->加载权重-->进行评估

In [7]:
def create_model_and_evaluate(cp_path): 
    '''
    创建一个新模型，并对新模型进行评估
    '''
    # 创建一个新模型
    model_new=create_model()
    # 从检查点加载权重
    model_new.load_weights(cp_path)
    # 评估该模型
    loss,acc=model_new.evaluate(test_images,test_labels)
    print('restored model accuracy: {:.3%}'.format(acc))

### 检查点回调选项

In [8]:
# 创建点路径
cp_path_id='traing_2/cp-{epoch:04d}.ckpt'
cp_dir_id=os.path.dirname(cp_path_id)

# 创建检点回调
cp_callback_id=tf.keras.callbacks.ModelCheckpoint(
                                                    cp_path_id,# 检查点路径
                                                    verbose=1,#
                                                    save_weights_only=True,# 只保存权重
                                                    period=5 # 每隔5个周期保存一次检查点，并设置唯一名称
                                                )
# 创建新模型
model_id=create_model()
# 拟合模型
model_id.fit(
            train_images,# 训练数据
            train_labels,epochs=50,# 训练标签
            callbacks=[cp_callback_id],# 回调
            validation_data=(test_images,test_labels), # 验证数据（测试数据，测试标签）
            verbose=0
            )



Epoch 00005: saving model to traing_2/cp-0005.ckpt

Epoch 00010: saving model to traing_2/cp-0010.ckpt

Epoch 00015: saving model to traing_2/cp-0015.ckpt

Epoch 00020: saving model to traing_2/cp-0020.ckpt

Epoch 00025: saving model to traing_2/cp-0025.ckpt

Epoch 00030: saving model to traing_2/cp-0030.ckpt

Epoch 00035: saving model to traing_2/cp-0035.ckpt

Epoch 00040: saving model to traing_2/cp-0040.ckpt

Epoch 00045: saving model to traing_2/cp-0045.ckpt

Epoch 00050: saving model to traing_2/cp-0050.ckpt


#### 查看最新的检查点

In [9]:
# 选择最新的检查点
last = tf.train.latest_checkpoint(cp_dir_id)
last

'traing_2\\cp-0050.ckpt'

#### 手动保存检查点

In [10]:
# 路径
cp_path_hand='./hand_save_cp/my_cp'

# 手动进行保存检查点
model_id.save_weights(cp_path_hand)

# 创建一个新模型，并对新模型进行评估
create_model_and_evaluate(cp_path_hand)

1000/1000 [==============================] - 0s 154us/step
restored model accuracy: 87.500%


### 保存整个模型(HDF5标准格式)
保存的内容：
- 权重值
- 模型配置(架构)
- 优化器配置

#### 保存完整模型

In [11]:
model =create_model()
model.fit(train_images,train_labels,epochs=5)

model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 1s 534us/step - loss: 1.1844 - acc: 0.6480
Epoch 2/5
1000/1000 [==============================] - 0s 141us/step - loss: 0.4000 - acc: 0.8950
Epoch 3/5
1000/1000 [==============================] - 0s 139us/step - loss: 0.2742 - acc: 0.9370
Epoch 4/5
1000/1000 [==============================] - 0s 141us/step - loss: 0.2090 - acc: 0.9440
Epoch 5/5
1000/1000 [==============================] - 0s 233us/step - loss: 0.1477 - acc: 0.9760


#### 加载完成模型

In [ ]:
new_model=keras.models.load_model('my_model.h5')
new_model.summary()

检查准确率

In [ ]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))